In [5]:
!pip install -U langchain-openai langchain_community langchain_anthropic langgraph

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.8/143.8 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.8/43.8 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.5/216.5 kB 9.0 MB/s eta 0:00:00


## Library

In [7]:
import os
from typing import TypedDict, Annotated
from langchain_core.messages import AnyMessage, SystemMessage, HumanMessage, ToolMessage
from langchain_core.tools import tool
from langchain_anthropic import ChatAnthropic
from langgraph.graph import StateGraph, END
from langgraph.prebuilt import ToolNode
# from langgraph.checkpoint import MemorySaver

In [9]:
from google.colab import userdata
os.environ["ANTHROPIC_API_KEY"] = userdata.get('ANTHROPIC_API_KEY')

## Agent Config

In [10]:
# For security, it's best to set your Anthropic API key as an environment variable.
# This class defines the structure of the state that will be passed between nodes in our graph.
class AgentState(TypedDict):
    """
    Represents the state of our agent.

    Attributes:
        messages: A list of messages in the conversation, managed by LangGraph.
    """
    messages: Annotated[list[AnyMessage], "The history of messages in the conversation."]


# Step 3: Define the Agent's Tools and Functions
# Tools are functions the agent can decide to call to perform actions.
@tool
def search(query: str):
    """Simulates a web search to find information."""
    # This is a simple mock tool. In a real application, this could
    # call a real search engine API.
    if "weather" in query.lower():
        return "It is sunny today in California."
    return "No relevant information found."

# A list of all tools the agent has access to.
tools = [search]
# A special node in LangGraph that executes the tools the model decides to call.
tool_node = ToolNode(tools)


# Step 4: Create the Agent Logic
# This is the core logic of the agent, where it decides what to do next.
# We are using Anthropic's Claude 3.5 Sonnet model.
model = ChatAnthropic(model="claude-3-5-sonnet-20240620", temperature=0).bind_tools(tools)

def call_model(state: AgentState):
    """
    The primary function that calls the LLM.
    This function is a node in our graph.
    """
    messages = state['messages']
    # Invoke the model with the current conversation history.
    response = model.invoke(messages)
    # Return the response to be added to the state.
    return {"messages": [response]}

def should_continue(state: AgentState):
    """
    A conditional function that decides the next step in the graph.
    This function is used for conditional edges.
    """
    last_message = state['messages'][-1]
    # If the model has made tool calls, we route to the 'tools' node.
    if last_message.tool_calls:
        return "tools"
    # Otherwise, we end the conversation.
    return END


# Step 5: Build the Graph
# The StateGraph is the core of the LangGraph application.
workflow = StateGraph(AgentState)

# Add the nodes to the graph.
workflow.add_node("agent", call_model)
workflow.add_node("tools", tool_node)

# Set the entry point for the graph.
workflow.set_entry_point("agent")

# Add the conditional edge. The graph will route to 'tools' or 'END'
# based on the output of the 'should_continue' function.
workflow.add_conditional_edges(
    "agent",
    should_continue,
    {
        "tools": "tools",
        "end": END,
    },
)

# Add a regular edge from the 'tools' node back to the 'agent' node.
# This creates the loop, allowing the agent to process tool results.
workflow.add_edge("tools", "agent")


# Step 6: Add Persistence
# A checkpointer allows the graph to be stopped and resumed.
# MemorySaver keeps the state in memory.
# checkpointer = MemorySaver()


# Step 7: Compile and Invoke the Agent
# Compile the graph into a runnable application.
app = workflow.compile()

# Define the initial conversation input.
# The 'thread_id' is used by the checkpointer to save/load the state.
config = {"configurable": {"thread_id": "1"}}
initial_input = {
    "messages": [HumanMessage(content="What is the weather today?")]
}

# Run the graph and stream the output.
print("\nAgent Invocation")
for s in app.stream(initial_input, config=config):
    print(s)
    print("--")

# You can also inspect the final state directly.
final_state = app.invoke(initial_input, config)
print("\nFinal Response")
print(final_state['messages'][-1].content)


Agent Invocation


BadRequestError: Error code: 400 - {'type': 'error', 'error': {'type': 'invalid_request_error', 'message': 'Your credit balance is too low to access the Anthropic API. Please go to Plans & Billing to upgrade or purchase credits.'}}